In [9]:
from huggingface_hub import create_repo
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from peft import PeftModel
import shutil

In [10]:
etiquetas = {
    0: "negativo",
    1: "neutral",
    2: "positivo"
}

modelo = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizador = AutoTokenizer.from_pretrained(modelo)
modelo_base = AutoModelForSequenceClassification.from_pretrained(
    modelo,
    num_labels=3,
    ignore_mismatched_sizes=True,
    id2label=etiquetas,
    label2id={v: k for k, v in etiquetas.items()}
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nlptown/bert-base-multilingual-uncased-sentiment and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([5]) in the checkpoint and torch.Size([3]) in the model instantiated
- classifier.weight: found shape torch.Size([5, 768]) in the checkpoint and torch.Size([3, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# Carga el modelo entrenado con LoRA
modelo_lora = PeftModel.from_pretrained(modelo_base, "./resultados/checkpoint-8760")

# Hace el merge de LoRA con el modelo base
modelo_final = modelo_lora.merge_and_unload() #type:ignore

# Guarda el modelo fusionado
modelo_final.save_pretrained("./modelo_sa")
tokenizador.save_pretrained("./modelo_sa")

('./modelo_sa\\tokenizer_config.json',
 './modelo_sa\\special_tokens_map.json',
 './modelo_sa\\vocab.txt',
 './modelo_sa\\added_tokens.json',
 './modelo_sa\\tokenizer.json')

In [12]:
repo_id = "SenaSoft/chdv-sentiment-analysis"

# Crear el repo en HF si no existe
create_repo(repo_id, exist_ok=True)

shutil.copy("./resultados/checkpoint-8760/README.md", "./modelo_sa/README.md")

# Subir el modelo fusionado
modelo_final.push_to_hub(repo_id)
tokenizador.push_to_hub(repo_id)

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...\Temp\tmphsvncpt7\model.safetensors:   9%|8         | 58.7MB /  669MB            

README.md: 0.00B [00:00, ?B/s]

CommitInfo(commit_url='https://huggingface.co/SenaSoft/chdv-sentiment-analysis/commit/425059fa16628bffdc92aa7e3fe6f84ffdd3b982', commit_message='Upload tokenizer', commit_description='', oid='425059fa16628bffdc92aa7e3fe6f84ffdd3b982', pr_url=None, repo_url=RepoUrl('https://huggingface.co/SenaSoft/chdv-sentiment-analysis', endpoint='https://huggingface.co', repo_type='model', repo_id='SenaSoft/chdv-sentiment-analysis'), pr_revision=None, pr_num=None)